In [12]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNNV2, RNN, RNNV12
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = './train_log/all-stage/'
logger.set_logger_dir(log_dir, action='d')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1103 17:53:53 @logger.py:74] Argv: /home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1007/jupyter/kernel-c5773769-f885-44ee-affa-4308d0f383e1.json


In [13]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

In [14]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [15]:
config.proportion = {'train': 0.7, 'val':0.3, 'test': 0.0}
config.stages = [2, 3, 4, 5, 6]
config.annotation_number = 20
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (3780, 16120), 'train': (2999, 13163), 'val': (1177, 5297)}

In [16]:
dm.get_imbalance_ratio()

,train,val,test
anterior midgut primordium,8.115502,7.654412,8.264706
brain primordium,12.096070,11.934066,12.309859
cellular blastoderm,9.063758,8.491935,8.947368
embryonic brain,8.401254,8.341270,8.379653
embryonic central nervous system,12.884259,11.134021,13.482759
embryonic dorsal epidermis,11.600840,10.653465,11.813559
embryonic hindgut,10.025735,9.603604,10.183432
embryonic midgut,5.694196,5.575419,5.654930
embryonic ventral epidermis,13.146226,11.934066,13.538462
embryonic/larval muscle system,12.096070,12.077778,12.170732


In [17]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = 
config.doubly_stochastic_lambda = 1e-3
config.read_time = 5
config.batch_size = 64

threshold = 0.4
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()

In [18]:
np.set_printoptions(formatter={'float_kind': lambda x: '%.2f' % x})
model = RNN(config, is_finetuning=False, 
              label_scale=dm.get_imbalance_ratio().train.values)
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-4), 
                                                                           (40, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=22, tower=[0])
Trainer(train_config).train()

[1103 17:54:01 @inference_runner.py:82] InferenceRunner will eval on an InputSource of size 18
[1103 17:54:01 @input_source.py:180] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1103 17:54:01 @training.py:90] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.


[1103 17:54:11 @base.py:212] Creating the session ...
[1103 17:54:13 @base.py:216] Initializing the session ...
[1103 17:54:13 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1103 17:54:14 @base.py:223] Graph Finalized.
[1103 17:54:14 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1103 17:54:15 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1103 17:54:16 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1103 17:54:16 @base.py:257] Start Epoch 1 ...


100%|##########|46/46[01:44<00:00, 0.44it/s]

[1103 17:56:00 @base.py:267] Epoch 1 (global_step 46) finished, time:104.45 sec.



  0%|          |0/18[00:00<?,?it/s]/home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|18/18[00:28<00:00, 0.59it/s]


...

[1103 18:32:49 @saver.py:90] Model saved to ./train_log/all-stage/model-966.
[1103 18:32:49 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.542
[1103 18:32:49 @monitor.py:362] QueueInput/queue_size: 49.999
[1103 18:32:49 @monitor.py:362] coverage: 2.9566
[1103 18:32:49 @monitor.py:362] learning_rate: 0.0001
[1103 18:32:49 @monitor.py:362] loss/value: 0.10964
[1103 18:32:49 @monitor.py:362] macro_auc: 0.94399
[1103 18:32:49 @monitor.py:362] macro_f1: 0.45075
[1103 18:32:49 @monitor.py:362] mean_average_precision: 0.72654
[1103 18:32:49 @monitor.py:362] micro_auc: 0.94365
[1103 18:32:49 @monitor.py:362] micro_f1: 0.46634
[1103 18:32:49 @monitor.py:362] one_error: 0.30556
[1103 18:32:49 @monitor.py:362] ranking_loss: 0.062439
[1103 18:32:49 @monitor.py:362] ranking_mean_average_precision: 0.7709
[1103 18:32:49 @monitor.py:362] training_ap: 0.60871
[1103 18:32:49 @monitor.py:362] training_auc: 0.92012
[1103 18:32:49 @group.py:42] Callbacks took 25.251 sec in total. D

100%|##########|46/46[01:22<00:00, 0.55it/s]

[1103 18:34:12 @base.py:267] Epoch 22 (global_step 1012) finished, time:82.52 sec.



100%|##########|18/18[00:25<00:00, 0.71it/s]


[1103 18:34:37 @saver.py:90] Model saved to ./train_log/all-stage/model-1012.
[1103 18:34:37 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.827
[1103 18:34:37 @monitor.py:362] QueueInput/queue_size: 50
[1103 18:34:37 @monitor.py:362] coverage: 3.3628
[1103 18:34:37 @monitor.py:362] learning_rate: 0.0001
[1103 18:34:37 @monitor.py:362] loss/value: 0.10494
[1103 18:34:37 @monitor.py:362] macro_auc: 0.94593
[1103 18:34:37 @monitor.py:362] macro_f1: 0.47601
[1103 18:34:37 @monitor.py:362] mean_average_precision: 0.74381
[1103 18:34:37 @monitor.py:362] micro_auc: 0.92501
[1103 18:34:37 @monitor.py:362] micro_f1: 0.47503
[1103 18:34:37 @monitor.py:362] one_error: 0.3724
[1103 18:34:37 @monitor.py:362] ranking_loss: 0.081982
[1103 18:34:37 @monitor.py:362] ranking_mean_average_precision: 0.72581
[1103 18:34:37 @monitor.py:362] training_ap: 0.6167
[1103 18:34:37 @monitor.py:362] training_auc: 0.92234
[1103 18:34:37 @group.py:42] Callbacks took 25.353 sec in total. DataP

In [14]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'EMA/QueueInput/queue_size:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/biased:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/local_step:0' shape=() dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bot